In [1]:
import argparse
import logging
import math
import os
import random
import shutil
import time
from collections import OrderedDict
import pdb

import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd


from dataset.cifar import DATASET_GETTERS
from utils import AverageMeter, accuracy

logger = logging.getLogger(__name__)
best_acc = 0
from numpy.linalg import inv
from numpy.linalg import det
from sklearn.metrics import mean_squared_error
from skimage.metrics import structural_similarity
import torch.nn as nn
from scipy import spatial

# Mu for unlabled loader is 1 currently

In [ ]:
#Save checkpoint funtion
def save_checkpoint(state, is_best, checkpoint, filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint,
                                               'model_best.pth.tar'))

#Setting the seed
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def get_cosine_schedule_with_warmup(optimizer,
                                    num_warmup_steps,
                                    num_training_steps,
                                    num_cycles=7./16.,
                                    last_epoch=-1):
    def _lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        no_progress = float(current_step - num_warmup_steps) / \
            float(max(1, num_training_steps - num_warmup_steps))
        return max(0., math.cos(math.pi * num_cycles * no_progress))

    return LambdaLR(optimizer, _lr_lambda, last_epoch)


def interleave(x, size):
    s = list(x.shape)
    return x.reshape([-1, size] + s[1:]).transpose(0, 1).reshape([-1] + s[1:])


def de_interleave(x, size):
    s = list(x.shape)
    return x.reshape([size, -1] + s[1:]).transpose(0, 1).reshape([-1] + s[1:])


# if args.seed is not None:
#         set_seed(args)

In [ ]:
parser = argparse.ArgumentParser(description='PyTorch FixMatch Training')
parser.add_argument('-f')
parser.add_argument('--gpu-id', default='1', type=int,
                    help='id(s) for CUDA_VISIBLE_DEVICES')
parser.add_argument('--num-workers', type=int, default=4,
                    help='number of workers')
parser.add_argument('--dataset', default='cifar10', type=str,
                    choices=['cifar10', 'cifar100'],
                    help='dataset name')
parser.add_argument('--num-labeled', type=int, default=40,
                    help='number of labeled data')
parser.add_argument("--expand-labels", action="store_true",
                    help="expand labels to fit eval steps")
parser.add_argument('--arch', default='wideresnet', type=str,
                    choices=['wideresnet', 'resnext'],
                    help='dataset name')
parser.add_argument('--total-steps', default=2**20, type=int,
                    help='number of total steps to run')
parser.add_argument('--eval-step', default=1024, type=int,
                    help='number of eval steps to run')
parser.add_argument('--start-epoch', default=0, type=int,
                    help='manual epoch number (useful on restarts)')
parser.add_argument('--batch-size', default=64, type=int,
                    help='train batchsize')
parser.add_argument('--lr', '--learning-rate', default=0.03, type=float,
                    help='initial learning rate')
parser.add_argument('--warmup', default=0, type=float,
                    help='warmup epochs (unlabeled data based)')
parser.add_argument('--wdecay', default=5e-4, type=float,
                    help='weight decay')
parser.add_argument('--nesterov', action='store_true', default=True,
                    help='use nesterov momentum')
parser.add_argument('--use-ema', action='store_true', default=True,
                    help='use EMA model')
parser.add_argument('--ema-decay', default=0.999, type=float,
                    help='EMA decay rate')
parser.add_argument('--mu', default=7, type=int,
                    help='coefficient of unlabeled batch size')
parser.add_argument('--lambda-u', default=1, type=float,
                    help='coefficient of unlabeled loss')
parser.add_argument('--T', default=1, type=float,
                    help='pseudo label temperature')
parser.add_argument('--threshold', default=0.95, type=float,
                    help='pseudo label threshold')
parser.add_argument('--out', default='result',
                    help='directory to output the result')
parser.add_argument('--resume', default='', type=str,
                    help='path to latest checkpoint (default: none)')
parser.add_argument('--seed', default=5, type=int,
                    help="random seed")
parser.add_argument("--amp", action="store_true",
                    help="use 16-bit (mixed) precision through NVIDIA apex AMP")
parser.add_argument("--opt_level", type=str, default="O1",
                    help="apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
                    "See details at https://nvidia.github.io/apex/amp.html")
parser.add_argument("--local_rank", type=int, default=-1,
                    help="For distributed training: local_rank")
parser.add_argument('--no-progress', action='store_true',
                    help="don't use progress bar")

args = parser.parse_args()

In [ ]:
def create_model(args):
    if args.arch == 'wideresnet':
        import models.wideresnet as models
        model = models.build_wideresnet(depth=args.model_depth,
                                        widen_factor=args.model_width,
                                        dropout=0,
                                        num_classes=args.num_classes)
    elif args.arch == 'resnext':
        import models.resnext as models
        model = models.build_resnext(cardinality=args.model_cardinality,
                                     depth=args.model_depth,
                                     width=args.model_width,
                                     num_classes=args.num_classes)
    logger.info("Total params: {:.2f}M".format(
        sum(p.numel() for p in model.parameters())/1e6))
    return model

if args.local_rank == -1:
    device = torch.device('cuda', args.gpu_id)
    args.world_size = 1
    args.n_gpu = torch.cuda.device_count()
else:
    torch.cuda.set_device(args.local_rank)
    device = torch.device('cuda', args.local_rank)
    torch.distributed.init_process_group(backend='nccl')
    args.world_size = torch.distributed.get_world_size()
    args.n_gpu = 1

args.device = device


In [ ]:
if args.dataset == 'cifar10':
    args.num_classes = 10
    if args.arch == 'wideresnet':
        args.model_depth = 28
        args.model_width = 2
    elif args.arch == 'resnext':
        args.model_cardinality = 4
        args.model_depth = 28
        args.model_width = 4

elif args.dataset == 'cifar100':
    args.num_classes = 100
    if args.arch == 'wideresnet':
        args.model_depth = 28
        args.model_width = 8
    elif args.arch == 'resnext':
        args.model_cardinality = 8
        args.model_depth = 29
        args.model_width = 64

In [ ]:
labeled_dataset, unlabeled_dataset, test_dataset = DATASET_GETTERS[args.dataset](
    args, './data')

In [ ]:
labeled_dataset.targets

In [ ]:
# labeled_dataset.data = np.load('/xace/d2/sumeet/Fix_Match_Data/Labeled_Dataset_Images_New_6.npy')
# labeled_dataset.targets = np.load('/xace/d2/sumeet/Fix_Match_Data/Labeled_Dataset_Labels_New_6.npy')

In [ ]:
# check_images = np.load('/xace/d2/sumeet/Fix_Match_Data/Labeled_Dataset_Images_New_6.npy')
# check_labels = np.load('/xace/d2/sumeet/Fix_Match_Data/Labeled_Dataset_Labels_New_6.npy')

# check_images = check_images[250:]
# check_labels = check_labels[250:]

In [ ]:
# count = 0
# for i in range(len(check_images)):
#     for j in range(len(unlabeled_dataset.data)):
#         if (check_images[i] == unlabeled_dataset.data[j]).all():
#             if check_labels[i] == unlabeled_dataset.targets[j]:
#                 count+=1
#                 break
#             break
        
# count/len(check_labels)        

In [ ]:
if args.local_rank == 0:
    torch.distributed.barrier()

train_sampler = RandomSampler if args.local_rank == -1 else DistributedSampler

In [ ]:
labeled_trainloader = DataLoader(
    labeled_dataset,
    sampler=train_sampler(labeled_dataset),
    batch_size=args.batch_size,
    num_workers=args.num_workers,
    drop_last=True)

#UnLabeled Train data loader
unlabeled_trainloader = DataLoader(
    unlabeled_dataset,
    sampler=train_sampler(unlabeled_dataset),
    batch_size=args.batch_size*args.mu,
#     batch_size=args.batch_size,
    num_workers=args.num_workers,
    drop_last=True)


#Test data loader
test_loader = DataLoader(
    test_dataset,
    sampler=SequentialSampler(test_dataset),
    batch_size=args.batch_size,
    num_workers=args.num_workers)

In [ ]:
if args.local_rank not in [-1, 0]:
    torch.distributed.barrier()

model = create_model(args)

if args.local_rank == 0:
    torch.distributed.barrier()

In [ ]:
# PATH = 'results/cifar10@250/checkpoint.pth.tar'

PATH = '/home/sumeet1/FixMatch-pytorch/results/cifar10@40.5/model_best.pth.tar'

checkpoint = torch.load(PATH)

In [ ]:
model.load_state_dict(checkpoint['state_dict'])
model.to(args.device)

In [ ]:
test_preds = torch.Tensor([])
test_labels = torch.Tensor([])
def test(args, test_loader, model, epoch):
    global test_preds
    global test_labels
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    end = time.time()

    if not args.no_progress:
        test_loader = tqdm(test_loader,
                           disable=args.local_rank not in [-1, 0])

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            data_time.update(time.time() - end)
            model.eval()

            inputs = inputs.to(args.device)
            targets = targets.to(args.device)
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, targets)

            prec1, prec5 = accuracy(outputs, targets, topk=(1, 5))
            losses.update(loss.item(), inputs.shape[0])
            top1.update(prec1.item(), inputs.shape[0])
            top5.update(prec5.item(), inputs.shape[0])
            batch_time.update(time.time() - end)
            end = time.time()
            test_preds = torch.cat((test_preds, outputs.cpu().detach()), 0)
            test_labels = torch.cat((test_labels, targets.cpu().detach()), 0)
            
            if not args.no_progress:
                test_loader.set_description("Test Iter: {batch:4}/{iter:4}. Data: {data:.3f}s. Batch: {bt:.3f}s. Loss: {loss:.4f}. top1: {top1:.2f}. top5: {top5:.2f}. ".format(
                    batch=batch_idx + 1,
                    iter=len(test_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                ))
        if not args.no_progress:
            test_loader.close()

    logger.info("top-1 acc: {:.2f}".format(top1.avg))
    logger.info("top-5 acc: {:.2f}".format(top5.avg))
    return losses.avg, top1.avg

In [ ]:
if args.use_ema:
    from models.ema import ModelEMA
    ema_model = ModelEMA(args, model, args.ema_decay)

In [ ]:
no_decay = ['bias', 'bn']
grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(
        nd in n for nd in no_decay)], 'weight_decay': args.wdecay},
    {'params': [p for n, p in model.named_parameters() if any(
        nd in n for nd in no_decay)], 'weight_decay': 0.0}]


optimizer = optim.SGD(grouped_parameters, lr=args.lr,
                          momentum=0.9, nesterov=args.nesterov)

In [ ]:
args.epochs = math.ceil(args.total_steps / args.eval_step)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(
        optimizer, args.warmup, args.total_steps)

In [ ]:
ema_model.ema.load_state_dict(checkpoint['ema_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
scheduler.load_state_dict(checkpoint['scheduler'])

In [ ]:
if args.amp:
    from apex import amp
    model, optimizer = amp.initialize(
        model, optimizer, opt_level=args.opt_level)

if args.local_rank != -1:
    model = torch.nn.parallel.DistributedDataParallel(
        model, device_ids=[args.local_rank],
        output_device=args.local_rank, find_unused_parameters=True)
    
model.zero_grad()

In [ ]:
def test(args, test_loader, model, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    end = time.time()

    if not args.no_progress:
        test_loader = tqdm(test_loader,
                           disable=args.local_rank not in [-1, 0])

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            data_time.update(time.time() - end)
            model.eval()

            inputs = inputs.to(args.device)
            targets = targets.to(args.device)
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, targets)

            prec1, prec5 = accuracy(outputs, targets, topk=(1, 5))
            losses.update(loss.item(), inputs.shape[0])
            top1.update(prec1.item(), inputs.shape[0])
            top5.update(prec5.item(), inputs.shape[0])
            batch_time.update(time.time() - end)
            end = time.time()
            if not args.no_progress:
                test_loader.set_description("Test Iter: {batch:4}/{iter:4}. Data: {data:.3f}s. Batch: {bt:.3f}s. Loss: {loss:.4f}. top1: {top1:.2f}. top5: {top5:.2f}. ".format(
                    batch=batch_idx + 1,
                    iter=len(test_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                ))
        if not args.no_progress:
            test_loader.close()

    logger.info("top-1 acc: {:.2f}".format(top1.avg))
    logger.info("top-5 acc: {:.2f}".format(top5.avg))
    return losses.avg, top1.avg

In [ ]:
labeled_iter = iter(labeled_trainloader)
unlabeled_iter = iter(unlabeled_trainloader)

model.train()

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [ ]:
model.block3.layer[3].conv2

In [ ]:
model.block1.layer[3].bn2.register_forward_hook(get_activation('bn2'))
output = model(next(labeled_trainloader.__iter__())[0].to(device))
resultant = activation['bn2']

In [ ]:
resultant.shape

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [ ]:
def gaussian_formula(pred,mu1,var1):
    form = (1/(math.sqrt(2*(math.pi)*var1))) * (math.exp(-(pred-mu1)**2/(2*var1)))
    return form

In [ ]:
model.train()
test_scores = []
label_ptr = 0
label_ptr1 = label_ptr + 50

fin_images = []
fin_labels = []


for epoch in range(args.start_epoch, 1024):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_x = AverageMeter()
    losses_u = AverageMeter()
    mask_probs = AverageMeter()
    if not args.no_progress:
        p_bar = tqdm(range(args.eval_step),
                     disable=args.local_rank not in [-1, 0])
    
    images_to_add = torch.Tensor([])
    labels_to_add = torch.LongTensor([])
    true_label = torch.LongTensor([])
    
    labeled_iter = iter(labeled_trainloader)
    labeled_iter = next(labeled_iter.__iter__())

    for batch_idx in range(1024):
        optimizer.zero_grad()
        
        try:
            
            inputs_x, targets_x = next(labeled_iter.__iter__())
                    
        except:
            if args.world_size > 1:
                labeled_epoch += 1
                labeled_trainloader.sampler.set_epoch(labeled_epoch)
            labeled_iter = iter(labeled_trainloader)
            inputs_x, targets_x =  next(labeled_iter.__iter__())

        try:
            (inputs_u_w, inputs_u_s,norm_x), unlab_labs = next(unlabeled_iter.__iter__())
        except:
            if args.world_size > 1:
                unlabeled_epoch += 1
                unlabeled_trainloader.sampler.set_epoch(unlabeled_epoch)
            unlabeled_iter = iter(unlabeled_trainloader)
            (inputs_u_w, inputs_u_s,norm_x), unlab_labs = next(unlabeled_iter.__iter__())


        batch_size = inputs_x.shape[0]
        inputs = interleave(
            torch.cat((inputs_x, inputs_u_w, inputs_u_s)), 2*args.mu+1).to(args.device)
        targets_x = targets_x.to(args.device)
        logits = model(inputs)
        logits = de_interleave(logits, 2*args.mu+1)
        logits_x = logits[:batch_size]
        logits_u_w, logits_u_s = logits[batch_size:].chunk(2)
        del logits

        Lx = F.cross_entropy(logits_x, targets_x, reduction='mean')

        pseudo_label = torch.softmax(logits_u_w.detach()/args.T, dim=-1)
        max_probs, targets_u = torch.max(pseudo_label, dim=-1)
        mask = max_probs.ge(args.threshold).float()

        
        
        unlab_labs = unlab_labs.to(device)
        
        pseudo_label1 = torch.softmax(logits_u_s.detach()/args.T, dim=-1)
        max_probs1, targets_u1 = torch.max(pseudo_label1, dim=-1)
        
#         Lu = (F.cross_entropy(logits_u_s, targets_u,
#                               reduction='none') * mask).mean()
        
     
##################################################################################################################################        
      ############################################## Gaussian Filter PART ##############################################################
        
        em_mask = np.zeros(len(inputs_u_w))
        for i in range(10):
            indexs = np.where(targets_u.cpu().detach()==i)
            images_to_add1 = norm_x[indexs]
            labels_to_add1 = targets_u[indexs]
            true_label1 = unlab_labs[indexs]
            p1 = max_probs[indexs].cpu().detach()
            p2 = max_probs1[indexs].cpu().detach()
            mu1 = np.mean(np.array(p2))
            var1 = np.var(np.array(p2))
            fin_p2 = np.ones(p2.shape)
            for i in range(3):
                fin_p2 = np.ones(p2.shape)
                for i in range(len(fin_p2)):
                    ans = gaussian_formula(p2[i],mu1,var1)
                    fin_p2[i]+=ans
                mu1 = np.mean(fin_p2)
                var1 = np.cov(fin_p2)   
                
#             fin_idx = np.where(fin_p2 >= np.mean(fin_p2))
            
            fin_idx = np.where(fin_p2 >= 0.95)
            
#             print(np.mean(fin_p2))
            em_mask_pointers = indexs[0][fin_idx]
            em_mask[em_mask_pointers]+=1
            
        for i in range(len(mask)):
            if mask[i]==0 and em_mask[i]==1:
                mask[i]+=1
                
        Lu = (F.cross_entropy(logits_u_s, targets_u,
                              reduction='none') * mask).mean()


    ########################################################################################################################
#     ###############################################Cosine Similarity########################################################
        if epoch>-1:
            if batch_idx >= label_ptr and batch_idx <= label_ptr1:
                with torch.no_grad():
                    model.relu.register_forward_hook(get_activation('relu'))
                    nu_output = model(inputs_x.to(device))
                    encoded_layer = activation['relu']


                    model.relu.register_forward_hook(get_activation('relu1'))
                    nu_output1 = model(inputs_u_w.to(device))
                    encoded_layer1 = activation['relu1']


                    r = nn.Flatten()
                    r1 = nn.Linear(8192,256)

                    l1  = r(encoded_layer.cpu().detach())
                    l2 = r1(l1)

                    l3  = r(encoded_layer1.cpu().detach())
                    l4 = r1(l3)

                    main_dict = {}
                    ssim_loss = {}
                    for i in range(10):
                        main_dict[i] = []
                        ssim_loss[i] = 0
                    for i in range(10):
                        dict1 = {}
                        ssim_loss_value = 0
                        labeled_labs = np.where(targets_x.cpu().detach()==i)[0]
                        to_compare_pseudo_labels = np.where(targets_u.cpu().detach()==i)[0]
                        for j in to_compare_pseudo_labels:
                            dict1[j] = 0

                        for k in labeled_labs:
                            for l in to_compare_pseudo_labels:
                                cosine_similarity = 1 - spatial.distance.cosine(l2[k], l4[l])
                                if cosine_similarity >= 0.7:
                                    dict1[l]+=1

                        for m in dict1:
                            if dict1[m]>=6:
                                main_dict[i].append(m)


                    for key in main_dict:
                        images_to_add = torch.cat((images_to_add, norm_x[main_dict[key]]), 0)
                        labels_to_add = torch.cat((labels_to_add, torch.Tensor(np.ones(len(main_dict[key]),dtype=int)*key)  ), 0)


##################################################################################################################################        
        
        loss = Lx + (args.lambda_u * Lu) 
#         loss = Lx 

        if args.amp:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()

        losses.update(loss.item())
        losses_x.update(Lx.item())
        losses_u.update(Lu.item())
        optimizer.step()
        scheduler.step()
        if args.use_ema:
            ema_model.update(model)
        model.zero_grad()

        mask_probs.update(mask.mean().item())

    if not args.no_progress:
        p_bar.close()

    if args.use_ema:
        test_model = ema_model.ema
    else:
        test_model = model
        
##################################################################################################################################    
    
    
    if len(labels_to_add)>0:
        for i in range(len(images_to_add)):
            flag = 0
            for j in range(len(labeled_dataset.data)):
                if (np.asarray(images_to_add[i])==labeled_dataset.data[j]).all():
                    flag+=1
            if flag==0:
                fin_images.append(np.asarray(images_to_add[i]))
                fin_labels.append(labels_to_add[i])
                
        print('Length of Fin_Images',len(fin_images))
        
        if len(np.unique(np.array(fin_labels)))==10:
            
            fin_images = np.array(fin_images)
            fin_labels = np.array(fin_labels)

            final_images_to_add = torch.Tensor([])
            final_labels_to_add = torch.LongTensor([])


            labels_to_add_pd = pd.DataFrame(fin_labels,dtype=int)
            labels_to_add_pd_min = labels_to_add_pd[0].value_counts().min()


            for i in range(10):
                reference = np.where(fin_labels==i)
                reference1 = reference[0][:labels_to_add_pd_min]
                final_images_to_add =   torch.cat((final_images_to_add,torch.Tensor(fin_images[reference1])),0)
                final_labels_to_add =   torch.cat((final_labels_to_add,torch.LongTensor(fin_labels[reference1])),0)

            final_images_to_add = np.array(final_images_to_add,dtype=np.uint8)
            final_labels_to_add = np.array(final_labels_to_add)
            
            
            
            #################### Checking for confounding error rate in every epoch#######################
            
            error_rate_count = 0
            for z in range(len(final_images_to_add)):
                for s in range(len(unlabeled_dataset.targets)):
                    if (np.asarray(final_images_to_add[z])==unlabeled_dataset.data[s]).all():
                        if final_labels_to_add[z] == unlabeled_dataset.targets[s]:
                            error_rate_count+=1
                            break
                        break
                            
            print('Error_Rate_Accuracy', error_rate_count/len(final_images_to_add))
            
            ###############################################################################################
            
            

            labeled_dataset.data = np.concatenate((labeled_dataset.data,final_images_to_add),axis = 0)
            labeled_dataset.targets = np.concatenate((labeled_dataset.targets,final_labels_to_add),axis = 0)

            print('labeled_dataset.data.shape',labeled_dataset.data.shape)
            
#             np.save('/xace/d2/sumeet/Fix_Match_Data/Labeled_Dataset_Images_New_6',labeled_dataset.data)
#             np.save('/xace/d2/sumeet/Fix_Match_Data/Labeled_Dataset_Labels_New_6',labeled_dataset.targets)

            labeled_trainloader = DataLoader(
            labeled_dataset,
            sampler=train_sampler(labeled_dataset),
            batch_size=args.batch_size,
            num_workers=args.num_workers,
            drop_last=True)
            
            fin_images = []
            fin_labels = []

#################################################################################################################################    
    print('Lx',Lx)
    print('Lu',Lu)
    print('Lambda:',args.lambda_u)
    test_loss, test_acc = test(args, test_loader, test_model, epoch)
    print('Test_Accuracy',test_acc)
    test_scores.append(test_acc)
    
    if epoch>-1:
        label_ptr+=50
        label_ptr1 = label_ptr + 50
